# Assignment 7 Clara Jones

In [ ]:
import sys
!{sys.executable} -m pip install dmba

In [14]:
%matplotlib inline
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
from sklearn.neural_network import MLPRegressor
import statsmodels.formula.api as sm
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

#dmba is module that book has

# Linear Regression

In [23]:
bh = pd.read_csv('BostonHousing.csv')
#bh = bh.iloc[0:1000]

predictors = ['CRIM', 'CHAS', 'NOX', 'RM', 'AGE', 'RAD', 'PTRATIO', 'LSTAT']
outcome = 'MEDV'

# partition data
X = pd.get_dummies(bh[predictors], drop_first=True)
y = bh[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

bh_lm = LinearRegression()
bh_lm.fit(train_X, train_y)

# print coefficients
print('intercept ', bh_lm.intercept_)
print(pd.DataFrame({'Predictor': X.columns, 'coefficient': bh_lm.coef_})) #coefficients are the Beta values

# print performance measures
regressionSummary(train_y, bh_lm.predict(train_X))

intercept  28.669029366461327
  Predictor  coefficient
0      CRIM    -0.082148
1      CHAS     2.958439
2       NOX   -13.081352
3        RM     3.773974
4       AGE     0.029100
5       RAD     0.121422
6   PTRATIO    -1.027816
7     LSTAT    -0.534890

Regression statistics

                      Mean Error (ME) : -0.0000
       Root Mean Squared Error (RMSE) : 4.8158
            Mean Absolute Error (MAE) : 3.3278
          Mean Percentage Error (MPE) : -3.7181
Mean Absolute Percentage Error (MAPE) : 17.4855


In [24]:
def RSMECalculate(Actual,Predict):
    return np.sqrt(np.sum((Predict - Actual)**2)/len(Actual))

In [25]:
bh_lm = LinearRegression()
bh_lm.fit(train_X, train_y)

bh_lm_pred = bh_lm.predict(train_X)
lm_train_RMSE = RSMECalculate(train_y,bh_lm_pred)

bh_lm_pred = bh_lm.predict(valid_X)
lm_valid_RMSE = RSMECalculate(valid_y, bh_lm_pred)

GlobalSummary = pd.DataFrame({'Train_RMSE':lm_train_RMSE,
                              'Valid_RMSE':lm_valid_RMSE},index = ['Regression'])
GlobalSummary

,Train_RMSE,Valid_RMSE
Regression,4.815784,5.426644


# MLP

In [26]:
bh_mlp = MLPRegressor(hidden_layer_sizes=(5), # means 1 hidden layer with 5 neurons, (2,4) means 1st layer has 2 neurons
                                               # and 2nd layer has 4 neurons
                       solver='lbfgs', max_iter=2000,
                      learning_rate = 'adaptive')
bh_mlp.fit(train_X, train_y)

bh_mlp_pred = bh_mlp.predict(train_X)
mlp_train_RMSE = RSMECalculate(train_y, bh_mlp_pred)

car_mlp_pred = bh_mlp.predict(valid_X)
mlp_valid_RMSE = RSMECalculate(valid_y, car_mlp_pred)

GlobalSummary = pd.DataFrame({'Train_RMSE':[lm_train_RMSE,mlp_train_RMSE],
                              'Valid_RMSE':[lm_valid_RMSE,mlp_valid_RMSE]},index = ['Regression','MLP'])
GlobalSummary 
# get diff result each time you run code bc MLP initially assigns random weights to each link

,Train_RMSE,Valid_RMSE
Regression,4.815784,5.426644
MLP,8.946008,9.560864


# Tuning MLP

In [19]:
# This code is basically creating a list of 15 one layered ANN ([1] - [15]) 
# and 100 two layered ANNs ([1,1] - [10,10])

PossibleNetStrct = []
PossibleNetString = []

for i in range(1,16):
    netStruct = [i]
    PossibleNetStrct.append(netStruct)
    PossibleNetString.append(str(netStruct))
    print('Network Structure:', netStruct)
    
for i in range(1,11):
    for j in range(1,11):
        netStruct = [i,j]
        PossibleNetStrct.append(netStruct)
        PossibleNetString.append(str(netStruct))
        print('Network Structure:', netStruct)

Network Structure: [1]
Network Structure: [2]
Network Structure: [3]
Network Structure: [4]
Network Structure: [5]
Network Structure: [6]
Network Structure: [7]
Network Structure: [8]
Network Structure: [9]
Network Structure: [10]
Network Structure: [11]
Network Structure: [12]
Network Structure: [13]
Network Structure: [14]
Network Structure: [15]
Network Structure: [1, 1]
Network Structure: [1, 2]
Network Structure: [1, 3]
Network Structure: [1, 4]
Network Structure: [1, 5]
Network Structure: [1, 6]
Network Structure: [1, 7]
Network Structure: [1, 8]
Network Structure: [1, 9]
Network Structure: [1, 10]
Network Structure: [2, 1]
Network Structure: [2, 2]
Network Structure: [2, 3]
Network Structure: [2, 4]
Network Structure: [2, 5]
Network Structure: [2, 6]
Network Structure: [2, 7]
Network Structure: [2, 8]
Network Structure: [2, 9]
Network Structure: [2, 10]
Network Structure: [3, 1]
Network Structure: [3, 2]
Network Structure: [3, 3]
Network Structure: [3, 4]
Network Structure: [3, 

In [20]:
i_train_X, i_valid_X, i_train_y, i_valid_y = train_test_split(train_X, train_y, test_size=0.2, random_state=1)

print('Train_X Shape: ', train_X.shape)
print('Train_Y Shape: ', train_y.shape)

print('Internal Train_X Shape: ', i_train_X.shape)
print('Internal Valid_X Shape: ', i_valid_X.shape)
print('Internal Train_y Shape: ', i_train_y.shape)
print('Internal Valid_y Shape: ', i_valid_y.shape)

Train_X Shape:  (303, 3)
Train_Y Shape:  (303,)
Internal Train_X Shape:  (242, 3)
Internal Valid_X Shape:  (61, 3)
Internal Train_y Shape:  (242,)
Internal Valid_y Shape:  (61,)


In [28]:
Results = pd.DataFrame(np.nan, index =PossibleNetString,
                      columns = ['R{}'.format(i) for i in range(1,6)])

for i,netStr in enumerate(PossibleNetStrct):
    RowName = PossibleNetString[i]
    for j in range(1,6):
        ColumnName = 'R' + str(j)
        
        bh_mlp = MLPRegressor(hidden_layer_sizes=netStr,
                       solver='lbfgs', max_iter=2000,
                      learning_rate = 'adaptive')
        bh_mlp.fit(i_train_X, i_train_y)

        bh_mlp_pred = bh_mlp.predict(i_valid_X)
        RMSE = np.sqrt(np.sum((bh_mlp_pred - i_valid_y)**2)/len(i_valid_y))
        
        Results.at[RowName, ColumnName] = RMSE
    print(netStr)

[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[1, 7]
[1, 8]
[1, 9]
[1, 10]
[2, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
[2, 8]
[2, 9]
[2, 10]
[3, 1]
[3, 2]
[3, 3]
[3, 4]
[3, 5]
[3, 6]
[3, 7]
[3, 8]
[3, 9]
[3, 10]
[4, 1]
[4, 2]
[4, 3]
[4, 4]
[4, 5]
[4, 6]
[4, 7]
[4, 8]
[4, 9]
[4, 10]
[5, 1]
[5, 2]
[5, 3]
[5, 4]
[5, 5]
[5, 6]
[5, 7]
[5, 8]
[5, 9]
[5, 10]
[6, 1]
[6, 2]
[6, 3]
[6, 4]
[6, 5]
[6, 6]
[6, 7]
[6, 8]
[6, 9]
[6, 10]
[7, 1]
[7, 2]
[7, 3]
[7, 4]
[7, 5]
[7, 6]
[7, 7]
[7, 8]
[7, 9]
[7, 10]
[8, 1]
[8, 2]
[8, 3]
[8, 4]
[8, 5]
[8, 6]
[8, 7]
[8, 8]
[8, 9]
[8, 10]
[9, 1]
[9, 2]
[9, 3]
[9, 4]
[9, 5]
[9, 6]
[9, 7]
[9, 8]
[9, 9]
[9, 10]
[10, 1]
[10, 2]
[10, 3]
[10, 4]
[10, 5]
[10, 6]
[10, 7]
[10, 8]
[10, 9]
[10, 10]


In [29]:
Results['Mean'] = Results[['R{}'.format(i) for i in range(1,6)]].mean(axis=1)
Results.head(10)

,R1,R2,R3,R4,R5,Mean
[1],4.428007,8.615529,8.616101,4.414270,8.616101,6.938002
[2],4.428032,7.417909,5.195428,5.195431,8.616101,6.170580
[3],4.457064,3.882993,3.841156,4.530696,3.720491,4.086480
[4],3.720312,4.004926,4.041972,3.862110,4.628601,4.051584
[5],4.006381,3.886217,3.820403,8.616101,5.195425,5.104906
[6],3.841662,3.827831,4.439692,3.936695,3.903625,3.989901
[7],3.955025,4.041640,3.873456,4.421695,3.829412,4.024246
[8],3.929771,3.650175,3.765989,3.908777,3.955541,3.842051
[9],3.846555,4.099209,4.103212,4.009574,3.865190,3.984748
[10],3.959073,3.840750,4.014119,4.426687,4.385769,4.125279


In [30]:
BM = Results.Mean == Results.Mean.min()
Results[BM]

,R1,R2,R3,R4,R5,Mean
"[10, 6]",4.051341,3.495082,3.792574,3.789839,3.863332,3.798434


In [31]:
SelectedNetStruc = [7,10]

Nets = []
Results= pd.DataFrame(np.nan,index = range(1,11),columns=['RSME','Random_State'])

for i in range(1,16):
    
    bh_mlp = MLPRegressor(hidden_layer_sizes=SelectedNetStruc,
                           solver='lbfgs', max_iter=2000,
                           learning_rate = 'adaptive',random_state = i) #control random state
    
    bh_mlp.fit(i_train_X, i_train_y)

    bh_mlp_pred = bh_mlp.predict(i_valid_X)
    RMSE = np.sqrt(np.sum((bh_mlp_pred - i_valid_y)**2)/len(i_valid_y))
    
    Nets.append(bh_mlp)
    Results.at[i,'RSME'] = RMSE
    Results.at[i,'Random_State'] = i

Results

,RSME,Random_State
1,3.788532,1.0
2,4.049078,2.0
3,3.911306,3.0
4,3.856881,4.0
5,3.746902,5.0
6,4.578033,6.0
7,4.115007,7.0
8,4.014214,8.0
9,4.670996,9.0
10,3.783325,10.0


In [32]:
SelectedNetStruc = [10,6]

Nets = []
Results= pd.DataFrame(np.nan,index = range(1,11),columns=['RSME','Random_State'])

for i in range(1,16):
    
    bh_mlp = MLPRegressor(hidden_layer_sizes=SelectedNetStruc,
                           solver='lbfgs', max_iter=2000,
                           learning_rate = 'adaptive',random_state = i) #control random state
    
    bh_mlp.fit(i_train_X, i_train_y)

    bh_mlp_pred = bh_mlp.predict(i_valid_X)
    RMSE = np.sqrt(np.sum((bh_mlp_pred - i_valid_y)**2)/len(i_valid_y))
    
    Nets.append(bh_mlp)
    Results.at[i,'RSME'] = RMSE
    Results.at[i,'Random_State'] = i

BM = Results.RSME == Results.RSME.min()
print('Best Validation Result: ', Results[BM].RSME)
Best_random_state = int(Results[BM].Random_State)
print('Best Random State: ' ,Best_random_state)

bh_mlp = MLPRegressor(hidden_layer_sizes=SelectedNetStruc,
                           solver='lbfgs', max_iter=2000,
                           learning_rate = 'adaptive',random_state = Best_random_state)

bh_mlp.fit(train_X, train_y)


bh_mlp_pred = bh_mlp.predict(train_X)
tuned_mlp_train_RMSE = np.sqrt(np.sum((bh_mlp_pred - train_y)**2)/len(train_y))

bh_mlp_pred = bh_mlp.predict(valid_X)
tuned_mlp_valid_RMSE = np.sqrt(np.sum((bh_mlp_pred - valid_y)**2)/len(valid_y))

GlobalSummary = pd.DataFrame({'Train_RMSE':[lm_train_RMSE,mlp_train_RMSE,tuned_mlp_train_RMSE],
                              'Valid_RMSE':[lm_valid_RMSE,mlp_valid_RMSE,tuned_mlp_valid_RMSE]},
                             index = ['Regression','MLP','MLP_tuned'])
GlobalSummary

Best Validation Result:  9    3.528638
Name: RSME, dtype: float64
Best Random State:  9


,Train_RMSE,Valid_RMSE
Regression,4.815784,5.426644
MLP,8.946008,9.560864
MLP_tuned,3.153137,4.926504


The tuned MLP model has a better performance than the Regression model